In [1]:
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format

train_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')

In [2]:
train_X.head(3)

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,0,2018-01-01,서귀포시,남원읍,291408.8970,18744.1310,219588.8570,529741.8840,53.0000,40.5710,2.3500,0.0000,3.3250
1,1,2018-01-01,제주시,노형동,1215420.2480,74964.8580,445735.3290,1736120.4350,38.5630,30.6550,-2.3000,0.0000,1.1000
2,2,2018-01-01,제주시,구좌읍,276220.0230,17542.1050,248104.1220,541866.2500,53.1880,40.7190,5.9000,0.0000,4.1000


In [3]:
train_y.head(3)

,id,교통량
0,0,76.2000
1,1,1291.1840
2,2,0.0000


In [4]:
train_y.describe()

,id,교통량
count,22448.0000,22448.0000
mean,11223.5000,374.6524
std,6480.3238,532.7097
min,0.0000,0.0000
25%,5611.7500,0.0000
50%,11223.5000,42.8245
75%,16835.2500,640.5845
max,22447.0000,2301.5000


In [5]:
train_y.kurtosis()

id    -1.2000
교통량    0.8404
dtype: float64

In [6]:
train_X.head(3)

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,0,2018-01-01,서귀포시,남원읍,291408.8970,18744.1310,219588.8570,529741.8840,53.0000,40.5710,2.3500,0.0000,3.3250
1,1,2018-01-01,제주시,노형동,1215420.2480,74964.8580,445735.3290,1736120.4350,38.5630,30.6550,-2.3000,0.0000,1.1000
2,2,2018-01-01,제주시,구좌읍,276220.0230,17542.1050,248104.1220,541866.2500,53.1880,40.7190,5.9000,0.0000,4.1000


In [7]:
train_X.describe().T

,count,mean,std,min,25%,50%,75%,max
id,22448.0000,11223.5000,6480.3238,0.0000,5611.7500,11223.5000,16835.2500,22447.0000
거주인구,22448.0000,317555.6997,298149.3833,10327.2590,94704.3245,221927.8885,410097.9040,1353560.5100
근무인구,22448.0000,35805.0281,40815.6840,1172.4480,12221.8195,21765.3370,40316.8758,272224.9300
방문인구,22448.0000,196133.7483,140600.5987,10788.3810,100005.0448,153158.8325,235928.0675,786387.1130
총 유동인구,22448.0000,549494.4760,461167.8723,22288.0880,222836.3663,389458.4280,641293.4193,2079277.8900
평균 속도,22448.0000,41.1016,8.7083,23.5450,34.2690,39.6515,49.1013,95.0000
평균 소요 시간,22448.0000,37.2217,12.9745,12.6670,27.8890,34.5000,46.1910,135.4000
평균 기온,22448.0000,13.5346,7.7496,-11.6000,7.6000,13.3000,19.7330,30.4000
일강수량,22448.0000,7.4018,28.8538,0.0000,0.0000,0.0000,1.5000,583.5000
평균 풍속,22448.0000,2.7382,1.4689,0.0000,1.7500,2.4000,3.4000,17.7000


In [8]:
full = train_X.merge(train_y, on='id')
full.set_index('id', inplace=True)
full.drop(['일자','읍면동명'], axis=1, inplace=True)

In [9]:
full['시도명'] = pd.factorize(full['시도명'])[0]

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(full, full['시도명']):
    s_train = full.loc[tr_idx]
    s_test = full.loc[te_idx]

In [11]:
x_train, y_train = s_train.drop('교통량', axis=1), s_train['교통량']
x_val, y_val = s_test.drop('교통량', axis=1), s_test['교통량']

In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
import xgboost as XGB

rf = RandomForestRegressor(random_state=156)
dt = DecisionTreeRegressor(random_state=156)
lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()
xgb = XGB.XGBRegressor(random_state=156)

rf.fit(x_train, y_train)
dt.fit(x_train, y_train)
lr.fit(x_train, y_train)
ridge.fit(x_train, y_train)
lasso.fit(x_train, y_train)
xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=156,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
rf_pred = rf.predict(x_val)
dt_pred = dt.predict(x_val)
lr_pred = lr.predict(x_val)
ridge_pred = ridge.predict(x_val)
lasso_pred = lasso.predict(x_val)
xgb_pred = xgb.predict(x_val)

from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_val, rf_pred))
print(mean_squared_error(y_val, dt_pred))
print(mean_squared_error(y_val, lr_pred))
print(mean_squared_error(y_val, ridge_pred))
print(mean_squared_error(y_val, lasso_pred))
print(mean_squared_error(y_val, xgb_pred))

9747.953804701032
19604.46923710824
195388.39208310342
195458.5125212764
195465.25363975647
12489.983347468782


선형회귀모델들은 효과가 없다.  
추가 EDA 시행 후 비선형 모델들로만 적용하여 성능 추가 테스트

In [14]:
set()

set()

In [15]:
full = train_X.merge(train_y, on='id')

In [16]:
full['시도명'] = full['시도명'].factorize()[0]

In [17]:
tmp = full.groupby(['읍면동명']).median()['교통량'].reset_index()
# full['읍면동별 평균 교통량'] = full.merge(on='')
tmp.columns = ['읍면동명',' 읍면동명별 교통량median']
full = full.merge(tmp, on='읍면동명')

In [18]:
full.head(3)

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,교통량,읍면동명별 교통량median
0,0,2018-01-01,0,남원읍,291408.8970,18744.1310,219588.8570,529741.8840,53.0000,40.5710,2.3500,0.0000,3.3250,76.2000,69.1000
1,76,2018-01-03,0,남원읍,287121.5060,27767.0750,181151.4350,496040.0160,52.3710,41.5140,3.5750,0.0000,3.9250,73.9710,69.1000
2,97,2018-01-04,0,남원읍,279036.1040,27045.9870,188198.6470,494280.7380,52.1430,41.5140,3.2000,1.6250,2.4500,70.2860,69.1000


In [19]:
full['date_월'] = full['일자'].apply(lambda x: x[5:7])

In [20]:
tmp = full.groupby('date_월')['교통량'].median().reset_index()
tmp.columns = ['date_월',' 월별 교통량median']
full = full.merge(tmp, on='date_월')

In [21]:
full.drop(['일자', '읍면동명', 'date_월'], axis=1, inplace=True)

In [22]:
full.set_index('id', inplace=True)

In [23]:
full = full.sort_values(by='id')

In [33]:
tmp = full[['거주인구', '근무인구','방문인구','총 유동인구', '평균 속도', '평균 소요 시간', '평균 기온', '일강수량','평균 풍속']]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
full[tmp.columns] = pd.DataFrame(scaler.fit_transform(tmp), columns=tmp.columns)

In [34]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(full, full['시도명']):
    s_train = full.loc[tr_idx]
    s_test = full.loc[te_idx]

In [36]:
x_train, y_train = s_train.drop('교통량', axis=1), s_train['교통량']
x_val, y_val = s_test.drop('교통량', axis=1), s_test['교통량']

In [39]:
from sklearn.ensemble import GradientBoostingRegressor

In [40]:
rf = RandomForestRegressor(random_state=156)
dt = DecisionTreeRegressor(random_state=156)
lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()
xgb = XGB.XGBRegressor(random_state=156)
gbm = GradientBoostingRegressor(random_state=156)

rf.fit(x_train, y_train)
dt.fit(x_train, y_train)
lr.fit(x_train, y_train)
ridge.fit(x_train, y_train)
lasso.fit(x_train, y_train)
xgb.fit(x_train, y_train)
gbm.fit(x_train, y_train)

rf_pred = rf.predict(x_val)
dt_pred = dt.predict(x_val)
lr_pred = lr.predict(x_val)
ridge_pred = ridge.predict(x_val)
lasso_pred = lasso.predict(x_val)
xgb_pred = xgb.predict(x_val)
gbm_pred = gbm.predict(x_val)

from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_val, rf_pred))
print(mean_squared_error(y_val, dt_pred))
print(mean_squared_error(y_val, lr_pred))
print(mean_squared_error(y_val, ridge_pred))
print(mean_squared_error(y_val, lasso_pred))
print(mean_squared_error(y_val, xgb_pred))
print(mean_squared_error(y_val, gbm_pred))

8055.646566553097
17287.485601449887
25043.565497056614
25044.431248859484
25040.16632080349
9006.522257998558
14350.862302826028
